In [23]:
pip install bilibili-api-python
pip install SpeechRecognition
pip install openai
pip install langchain
pip install unstructured
pip install tiktoken

SyntaxError: invalid syntax (1942025046.py, line 1)

In [23]:
import asyncio
from bilibili_api import video,HEADERS
import httpx
import os

In [25]:
# 下载视频/音频 函数
async def download_url(url: str, out: str, info: str):
    # 下载函数
    async with httpx.AsyncClient(headers=HEADERS) as sess:
        resp = await sess.get(url)
        length = resp.headers.get('content-length')
        with open(out, 'wb') as f:
            process = 0
            for chunk in resp.iter_bytes(1024):
                if not chunk:
                    break

                process += len(chunk)
                print(f'下载 {info} {process} / {length}')
                f.write(chunk)

In [30]:
# 实例化 Video 类
v = video.Video(bvid="BV1xj411X7Ct")
# 获取信息
# info = await v.get_info()
# 打印信息
# print(info)
# 获取视频下载链接
download_url_data = await v.get_download_url(0)
# print(download_url_data)
# # 解析视频下载信息
detecter = video.VideoDownloadURLDataDetecter(data=download_url_data)
streams = detecter.detect_best_streams()
# print(detecter)
# print(streams)
print(streams)
print(streams[1].url)
await download_url(streams[1].url, "audio_temp.m4s", "音频流")
# # 有 MP4 流 / FLV 流两种可能
# if detecter.check_flv_stream() == True:
#     # FLV 流下载
#     await download_url(streams[0].url, "flv_temp.flv", "FLV 音视频流")
#     # 转换文件格式
#     os.system(f'{FFMPEG_PATH} -i flv_temp.flv video.mp4')
#     # 删除临时文件
#     os.remove("flv_temp.flv")
# else:
#     # MP4 流下载
#     await download_url(streams[0].url, "video_temp.m4s", "视频流")
#     await download_url(streams[1].url, "audio_temp.m4s", "音频流")
#     # 混流
#     os.system(f'{FFMPEG_PATH} -i video_temp.m4s -i audio_temp.m4s -vcodec copy -acodec copy video.mp4')
#     # 删除临时文件
#     os.remove("video_temp.m4s")
#     os.remove("audio_temp.m4s")

# print('已下载为：video.mp4')

[VideoStreamDownloadURL(url='https://xy106x87x100x133xy.mcdn.bilivideo.cn:8082/v1/resource/1164560587_x2-1-100023.m4s?agrr=0&build=0&buvid=9a5750e4-1c04-11ee-aa89-0e797ab632bb&bvc=vod&bw=30860&cdnid=11147&deadline=1688658859&e=ig8euxZM2rNcNbdlhoNvNC8BqJIzNbfqXBvEqxTEto8BTrNvN0GvT90W5JZMkX_YN0MvXg8gNEV4NC8xNEV4N03eN0B5tZlqNxTEto8BTrNvNeZVuJ10Kj_g2UB02J0mN0B5tZlqNCNEto8BTrNvNC7MTX502C8f2jmMQJ6mqF2fka1mqx6gqj0eN0B599M%3D&gen=playurlv2&logo=80000000&mid=0&nbs=1&nettype=0&oi=2029351492&orderid=0%2C3&os=bcache&platform=pc&sign=ed6350&traceid=trRkjgXcQdxfdz_0_e_N&uipk=5&uparams=e%2Cuipk%2Cnbs%2Cdeadline%2Cgen%2Cos%2Coi%2Ctrid%2Cmid%2Cplatform&upsig=5d76337f09dd79133137b09c2dafabdf', video_quality=<VideoQuality._480P: 32>, video_codecs=<VideoCodecs.AV1: 'av01'>), AudioStreamDownloadURL(url='https://xy114x235x252x21xy.mcdn.bilivideo.cn:8082/v1/resource/1164560587_nb3-1-30280.m4s?agrr=0&build=0&buvid=9a5750e4-1c04-11ee-aa89-0e797ab632bb&bvc=vod&bw=20312&cdnid=11147&deadline=1688658859&e=ig8euxZM

In [31]:
# m4s 转换成 wav格式
# 需要提前下载好 brew install ffmpeg 处理音频文件格式
import subprocess

# 指定M4S文件路径
m4s_file = "./audio_temp.m4s"

# 指定输出的音频文件路径（WAV格式）
audio_file = "audio_temp.wav"
# ffmpeg报错找不到路径一直报错 因此使用完整路径指定FFmpeg
ffmpeg_path = "/opt/homebrew/bin/ffmpeg"  # 替换为你的FFmpeg路径

# 使用ffmpeg提取音频轨道
subprocess.call([ffmpeg_path, "-i", m4s_file, "-vn", "-acodec", "pcm_s16le", "-ar", "44100", "-ac", "2", audio_file])


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --dis

0

In [53]:
# 使用google 语音识别
# SpeechRecognition 语音识别
# brew install flac 无损音频压缩编解码器  recognize_google方法中会进行执行 
# 如果用brewa install 安装成功运行代码报错：让你用apt-get进行安装 到/usr/local/bin 目录下 执行下面命令
# ln -s /opt/homebrew/bin/flac /usr/local/bin/flac

import speech_recognition as sr

# 创建一个Recognizer对象
r = sr.Recognizer()
audio_file = "audio_temp.wav"
# 使用Recognizer对象打开音频文件
with sr.AudioFile(audio_file) as source:
    # 将音频文件读取为音频数据
    audio = r.record(source)
    # print(audio)
    try:
        # 使用Google Web Speech API进行语音识别
        text = r.recognize_google(audio, language="zh-CN")
        # with open('audio_transcript.text', 'w', encoding='utf-8') as text_file:
        #     text_file.write(text)
        print("转换结果：", text)
    except sr.UnknownValueError:
        print("无法识别音频中的语音")
    except sr.RequestError as e:
        print("无法连接到Google Web Speech API；错误原因：", str(e))


转换结果： 27年前四个女高中生的随手一谈却不经意间成了几代人的毕业生取没有录音棚他们最真实最纯真的歌声唱出一段时光森两青春或许只有郑青春的他们能把毕业于离别写的如此神可让无数人感同身受非生东路的日子就是高中生王源用了不到一天的时间普显然后在18岁生日的前一天拉着11位同学录取而成它们用歌声将不舍与感伤永远铭刻在记忆里成了一代人的回忆ocbc双方均须同在出30两位十几岁的小女生一起将被卡的紧张对分离的不舍谱写成一首充满回忆与憧憬的歌曲两年后这首歌在中国好歌曲上获得了导师的一致好评也成为了无数人的毕业艰巨青春是一首歌正值青春的他的是这首歌里面最动听的旋律或许这才是这几首歌成为经典的原因吧那首歌是你的毕业惊喜欢迎留言咱们下棋的人


In [34]:
with open('transcript——bilibili.text', 'w', encoding='utf-8') as text_file:
    text_file.write(text)


In [35]:
cat transcript——bilibili.text

27年前四个女高中生的随手一谈却不经意间成了几代人的毕业生取没有录音棚他们最真实最纯真的歌声唱出一段时光森两青春或许只有郑青春的他们能把毕业于离别写的如此神可让无数人感同身受非生东路的日子就是高中生王源用了不到一天的时间普显然后在18岁生日的前一天拉着11位同学录取而成它们用歌声将不舍与感伤永远铭刻在记忆里成了一代人的回忆ocbc双方均须同在出30两位十几岁的小女生一起将被卡的紧张对分离的不舍谱写成一首充满回忆与憧憬的歌曲两年后这首歌在中国好歌曲上获得了导师的一致好评也成为了无数人的毕业艰巨青春是一首歌正值青春的他的是这首歌里面最动听的旋律或许这才是这几首歌成为经典的原因吧那首歌是你的毕业惊喜欢迎留言咱们下棋的人

In [36]:
from langchain.document_loaders import TextLoader

In [37]:
loader = TextLoader('./transcript——bilibili.text')
docs = loader.load()

In [38]:
print(f'you have {len(docs)} documents in your data')
print(f'there are {len(docs[0].page_content)} characters in your document')

you have 1 documents in your data
there are 310 characters in your document


In [39]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = text_splitter.split_documents(docs)
print(f'You have {len(split_docs)} split documents')

You have 1 split documents


In [42]:
import os 
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate

In [50]:
prompt_template = """写出以下内容的简洁摘要:
{text}
使用中文进行描述:"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_summarize_chain(llm,chain_type='map_reduce',verbose=True, map_prompt=PROMPT, combine_prompt=PROMPT)
input_docs = split_docs
chain.run(input_documents=input_docs)



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
写出以下内容的简洁摘要:
27年前四个女高中生的随手一谈却不经意间成了几代人的毕业生取没有录音棚他们最真实最纯真的歌声唱出一段时光森两青春或许只有郑青春的他们能把毕业于离别写的如此神可让无数人感同身受非生东路的日子就是高中生王源用了不到一天的时间普显然后在18岁生日的前一天拉着11位同学录取而成它们用歌声将不舍与感伤永远铭刻在记忆里成了一代人的回忆ocbc双方均须同在出30两位十几岁的小女生一起将被卡的紧张对分离的不舍谱写成一首充满回忆与憧憬的歌曲两年后这首歌在中国好歌曲上获得了导师的一致好评也成为了无数人的毕业艰巨青春是一首歌正值青春的他的是这首歌里面最动听的旋律或许这才是这几首歌成为经典的原因吧那首歌是你的毕业惊喜欢迎留言咱们下棋的人
使用中文进行描述:

> Finished chain.


> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
写出以下内容的简洁摘要:

27年前，四位女高中生在没有录音棚的情况下，用最真实最纯真的歌声，将不舍与感伤谱写成一首充满回忆与憧憬的歌曲，两年后，这首歌在中国好歌曲上获得了导师的一致好评，成为无数人毕业艰巨的青春经典。
使用中文进行描述:

> Finished chain.

> Finished chain.

> Finished chain.


'\n27年前，四位女高中生用最真实的歌声，谱写出一首充满回忆与憧憬的歌曲，两年后，这首歌获得了导师的一致好评，成为毕业艰巨时的青春经典。'